# General Workflow for Modulus Projects

1. Initialize Hydra using the Modulus main decorator to read in the configuration YAML.

2. Load necessary data if needed or define the geometry of the system if needed.

3. Create any Nodes required, such as your neural network model.

4. Create a training domain object.

5. Create constraints and add each to the domain.

6. Create any inferencers, validators or monitors needed.

7. Initialize a solver with the populated training domain.

8. Run the solver, beginning optimization.

Let's see an example of approximating a solution to given differential equation and boundary conditions with Modulus:

### 1. Load Hydra

[Hydra configuration package](https://docs.nvidia.com/deeplearning/modulus/user_guide/features/configuration.html) is built into the heart of Modulus. It allows the easy control over various hyperparameters using YAML files. In Modulus, Hydra is the first component to be initialized, and it has direct influence on all component levels inside of Modulus.

Here is the content of config.yaml file:

```yaml
defaults :
  - modulus_default
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

scheduler:
  decay_rate: 0.95
  decay_steps: 200

save_filetypes : "vtk,npz"

training:
  rec_results_freq : 1000
  rec_constraint_freq: 1000
  max_steps : 5000
```
You can use the config file to change a variety of training parameters like the learning rate, optimizer, decay_rate, decay_steps, etc. The results saveing frequency, constraints saving frequency and max_steps can also be defined. The modulus_default configs setting is used here you can also define more custom configs as well.

For running Modulus in Jupyter notebook, we can load config with:

In [ ]:
import modulus
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.hydra.config import ModulusConfig

cfg = compose(config_path="./", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

### 2. Define the Geometry

For this purely physics case, there is no external training data. Instead, Instead, we will create some geometry that will be used to sample various collocation points to impose the  PDE loss and boundary loss. Modulus has several geometry objects including 1D shapes like `Point1D` and `Line1D` and 3D shapes like `Torus`, `Tetrahedron` etc. Here, we use the `Line1D` object:

In [ ]:
from sympy import Symbol
from modulus.sym.geometry.primitives_1d import Line1D

# make geometry
x = Symbol("x")
geo = Line1D(0, 1)

Once the geometry object has been instantiated, you can sample the points in that geometry object using methods like [`sample_boundary`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.geometry.html#modulus.geometry.geometry.Geometry.sample_boundary) and [`sample_interior`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.geometry.html#modulus.geometry.geometry.Geometry.sample_interior) to see what is being sampled. 

In [ ]:
samples = geo.sample_boundary(5)
print("Boundary Samples", samples)

samples = geo.sample_interior(5)
print("Interior Samples", samples)

### 3. Create Nodes
Before setting up the nodes, let's first define the PDE for this problem. For this simple problem, a simple PDE can be set up as below. Modulus also contains several common pre-defined PDEs to choose from, such as Navier-Stokes, Linear Elasticity, Advection Diffusion, Wave Equations, etc.

The PDE class enables us to write equations symbolically in SymPy, so that we can quickly write equations in the most natural way. The SymPy equations are converted to PyTorch expressions in the back-end and can also be printed out to ensure correct implementation.

In [ ]:
from sympy import Symbol, Number, Function
from modulus.sym.eq.pde import PDE

class CustomPDE(PDE):
    def __init__(self, f=1.0):
        # coordinates
        x = Symbol("x")

        # make input variables
        input_variables = {"x": x}

        # make u function
        u = Function("u")(*input_variables)

        # source term
        if type(f) is str:
            f = Function(f)(*input_variables)
        elif type(f) in [float, int]:
            f = Number(f)

        # set equations
        self.equations = {}
        self.equations["custom_pde"] = (
            u.diff(x, 2) - f
        )  # "custom_pde" key name will be used in constraints

Now, we are going to create the nodes required for our problem. The word "nodes" here doesn't mean "compute nodes" or "login nodes" you might be familiar with in high-performance computing (HPC) environments. Instead, nodes in Modulus are used to represent components that will be executed in the forward pass during the training. These include the neural network itself and any equations used to formulate the PDE loss functions.

In [ ]:
from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.key import Key

eq = CustomPDE(f=1.0)

# Define fully connected neural network with 3 layers and 32 neurons in each layer. Takes "x" as input variable, "u" as input variable

u_net = FullyConnectedArch(
    input_keys=[Key("x")], output_keys=[Key("u")], nr_layers=3, layer_size=32
)

nodes = eq.make_nodes() + [u_net.make_node(name="u_network")]

### 4. Create a training domain object

The `Domain` includes constraints (such as geometric areas or volumes within which the equations of the problem are solved and the equations governing the physical phenomena, and boundary conditions) and additional components (such as inferencers, validators, and monitors) needed in the training process. Once user defins constraints, they are added to the Domain to create a collection of training objectives. Then, Domain and the configs are passed to neural network as inputs.

In [ ]:
from modulus.sym.domain import Domain

# make domain
domain = Domain()

In this physics-driven problem, constraints are the boundary conditions and equation residuals ("residuals" refer to the difference between the outputs of the neural network and the exact values dictated by the governing physical equations). Our goal is to satisfy the boundary conditions exactly, and ideally make the PDE residuals go to 0. Constraints can be specified using classes like [`PointwiseBoundaryConstrant`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.domain.constraint.html#modulus.domain.constraint.continuous.PointwiseBoundaryConstraint) and [`PointwiseInteriorConstraint`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.domain.constraint.html#modulus.domain.constraint.continuous.PointwiseInteriorConstraint). Later, a loss function (by default L2) will be generated based on these constraints and minimized by the optimizer.

To generate boundary conditions, we need to sample the points on the boundary/surface of our geometry, specify the nodes we would like to evaluate on these points, and then assign values to them as needed.

To generate a boundary condition, PointwiseBoundaryConstraint class will sample the entire boundary of our geometry, i.e. both endpoints of the 1D line. 

To solve the PDE equation we defined, PointwiseInteriorConstraint class will sample points in the interior of the geometry and the PDE is enforced on all the points in the interior.

In [ ]:
from modulus.sym.domain.constraint import PointwiseBoundaryConstraint, PointwiseInteriorConstraint

# bcs
bc = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"u": 0},   # the desired values of the boundary condition
    batch_size=2,      # the number of points to sample on each boundary
)
domain.add_constraint(bc, "bc")

# interior
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"custom_pde": 0},   # the desired solution of the PDE 
    batch_size=100,  
    bounds={x: (0, 1)},   # the range for sampling the values for variables
)
domain.add_constraint(interior, "interior")

### 5. Create any Inferencers, Validators or Monitors

Here, we will create an inferencer to visualize results. It will provide the predicted value of the output variable "u" for 100 linearly spaced points in the range [0, 1] of the input variable "x".

In [ ]:
import numpy as np
from modulus.sym.domain.inferencer import PointwiseInferencer

# add inferencer
inference = PointwiseInferencer(
    nodes=nodes,
    invar={"x": np.linspace(0, 1.0, 100).reshape(-1,1)},
    output_names=["u"],
)
domain.add_inferencer(inference, "inf_data")

### 6. Initialize and Run a Solver:
Here, we will initialize a solver with the Domain we just created and other configurations that define the optimizer choices using Solver class. Then, run the solver, begining optimization: 

In [ ]:
# to make the logging work in the jupyter cells
# execute this cell only once
import logging
logging.getLogger().addHandler(logging.StreamHandler())

In [ ]:
import os
from modulus.sym.solver import Solver

# optional set appropriate GPU in case of multi-GPU machine
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### 7. Visualizing the Results

The results can be plotted with `matplotlib`. The inferencer/validator writes the data to disk which we will use to make the plots. The output format can be modified in the `save_filetypes` in config.yaml file. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
data = np.load('./outputs/inferencers/inf_data.npz', allow_pickle=True)
data = np.atleast_1d(data.f.arr_0)[0]
plt.figure()
x = data['x'].flatten()
pred_u = data['u'].flatten()
plt.plot(np.sort(x), pred_u[np.argsort(x)], label='Neural Solver')
plt.plot(np.sort(x), 0.5*(np.sort(x)*(np.sort(x)-1)), label='(1/2)(x-1)x')
x_np = np.array([0., 1.])
u_np = 0.5*(x_np-1)*x_np
plt.scatter(x_np, u_np, label='BC')
plt.legend()
plt.show()

Finish! Well down!